In [22]:
!pip install Cython
%load_ext Cython
import numpy as np


SyntaxError: invalid syntax (<ipython-input-22-811b06197572>, line 4)

# cdef & cpdef

In [3]:
def function(x, y):
    return x**y

In [8]:
%timeit function(5, 6)

452 ns ± 8.41 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [ ]:
%%cython

cdef cdef_funtion(x, y):
    return x**y



In [5]:
cdef_funtion(5, 6)

NameError: name 'cdef_funtion' is not defined

Cdef functions are not callable from non cython cells.

In [6]:
%%cython

cpdef cpdef_function(x,y):
    return x**y

In [7]:
%timeit cpdef_function(5,6)

452 ns ± 8.06 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


Not really faster. So introduce datatypes!

In [9]:
%%cython

cpdef int cpdef_dtypes_function(int x, int y):
    return x**y

In [10]:
%timeit cpdef_dtypes_function(5,6)

115 ns ± 2.46 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


# Cython and Numpy Arrays

In [90]:
%%cython
import numpy as np

arr = np.arange(9, dtype=np.dtype("i")).reshape((3, 3))
print(arr)

cdef int [:, :] arr_view = arr
print(np.asarray(arr_view))

[[0 1 2]
 [3 4 5]
 [6 7 8]]
[[0 1 2]
 [3 4 5]
 [6 7 8]]


In [170]:
%%cython 
import numpy as np

cdef int [:, :] arr_view = np.arange(9, dtype=np.dtype("i")).reshape((3, 3))
print(np.asarray(arr_view))
print(np.asarray(arr_view[1,1]))
arr_view[1,1] = 10
print(np.asarray(arr_view[1,1]))
arr_view[:,:] = 5
print(np.asarray(arr_view))


[[0 1 2]
 [3 4 5]
 [6 7 8]]
4
10
[[5 5 5]
 [5 5 5]
 [5 5 5]]


## Looping over arrays

In [150]:
x = np.arange(100, dtype=np.dtype("i")).reshape((10, 10))
y = np.full((10,10), 2, dtype=np.dtype("i"))

In [134]:
def array_op(x,y):
    result = np.zeros(x.shape)
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            result[i,j] = x[i,j] * y[i,j] + i
    return result
    

In [138]:
%timeit result = array_op(x,y)

295 µs ± 7.75 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [139]:
%%cython 
import numpy as np

cpdef cdef_array_op(x, y):
    result = np.zeros(x.shape)
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            result[i,j] = x[i,j] * y[i,j] + i
    return result

In [140]:
%timeit cdef_array_op(x,y)

283 µs ± 12.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [166]:
%%cython 
import numpy as np

cpdef int[:,:] typed_array_op(int[:,:] x, int[:,:] y):
    cdef int[:,:] result = np.zeros((x.shape[0], x.shape[1]), dtype = np.dtype("i"))
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            result[i,j] = x[i,j] * y[i,j] + i
    return result

In [167]:
%timeit typed_array_op(x,y)

7.74 µs ± 104 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [168]:
%%cython 
import numpy as np
cimport cython

@cython.boundscheck(False)
@cython.wraparound(False)
cpdef int[:,:] typed_unsafe_array_op(int[:,:] x, int[:,:] y):
    cdef int[:,:] result = np.zeros((x.shape[0], x.shape[1]), dtype = np.dtype("i"))
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            result[i,j] = x[i,j] * y[i,j] + i
    return result


In [169]:
%timeit typed_unsafe_array_op(x,y)

7.66 µs ± 69.5 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


# Cython and Pandas

We downloaden eerst wat data...

In [ ]:
!curl -L -c cookies.txt 'https://docs.google.com/uc?export=download&id=151gCztjHR_D2uIoebxfi52DZWGLabOQd' | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1/p' > confirm.txt    
!curl -L -b cookies.txt -o 'weatherAUS.zip' 'https://docs.google.com/uc?export=download&id=151gCztjHR_D2uIoebxfi52DZWGLabOQd&confirm='$(<confirm.txt)
!unzip weatherAUS.zip
!rm -f confirm.txt cookies.txt weatherAUS.zip